# Local Alignment
Name: Yanfang Guo
Department : MACS
Email: Yanfang.Guo@vub.be

## Read the data and initialize it.
The job is actually the same as global alignment, so I will skip the explaination of this part

In [30]:
from Matrix import *
from read_fasta import *
E = -4
I = -6 # affine gap
seq_list = read_fasta("protein-sequences.fasta")
seq_list2 = read_fasta("WW-sequence.fasta")

mat1 = Matrix("BLOSUM62.txt")


The slen1 and slen2 are the length of the seq1 and seq2

In [31]:
seq1 =seq_list[0]
seq2= seq_list2[0]
slen1 = len(seq1)
slen2 = len(seq2)
print(seq1)
print(seq2)

MDPGQQPPPQPAPQGQGQPPSQPPQGQGPPSGPGQPAPAATQAAPQAPPAGHQIVHVRGDSETDLEALFNAVMNPKTANVPQTVPMRLRKLPDSFFKPPEPKSHSRQASTDAGTAGALTPQHVRAHSSPASLQLGAVSPGTLTPTGVVSGPAATPTAQHLRQSSFEIPDDVPLPAGWEMAKTSSGQRYFLNHIDQTTTWQDPRKAMLSQMNVTAPTSPPVQQNMMNSASGPLPDGWEQAMTQDGEIYYINHKNKTTSWLDPRLDPRFAMNQRISQSAPVKQPPPLAPQSPQGGVMGGSNSNQQQQMRLQQLQMEKERLRLKQQELLRQAMRNINPSTANSPKCQELALRSQLPTLEQDGGTQNPVSSPGMSQELRTMTTNSSDPFLNSGTYHSRDESTDSGLSMSSYSVPRTPDDFLNSVDEMDTGDTINQSTLPSQQNRFPDYLEAIPGTNVDLGTLEGDGMNIEGEELMPSLQEALSSDILNDMESVLAATKLDKESFLTWL
VPLPAGWEMAKTSSGQRYFLNHIDQTTTWQDPRK


## Initialize the score_mat and direc_mat
- This part is very similar to that in global alignment
- the values in first row and column of the score_mat are set to 0

In [32]:
score_mat = []
direc_mat = []
for i in range(len(seq1) + 1):
    direc_mat.append([[0, 0, 0,I,I] for i in range(len(seq2) + 1)])

for i in range(len(seq1) + 1):
    score_mat.append([0 for i in range(len(seq2) + 1)])


def print_mat2(alist):
    for i in alist:
        for j in i:
            print('{0:4d}'.format(j), end=" ")
        print()


def print_mat3(alist,n=3):
    for i in alist:
        for j in i:
            print(j[0:n],end='')
        print()

## Calculate the Score according to the formula
![](https://ww2.sinaimg.cn/large/006tNbRwgy1fdo45gbte9j30hq06ydgm.jpg)

In [33]:
def local_score(mat):
    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):

            t1 = score_mat[i - 1][j - 1] + mat.get(seq1[i - 1], seq2[j - 1])


            t2 =score_mat[i][j-1] +direc_mat[i][j-1][4]



            t3 = score_mat[i-1][j] + direc_mat[i-1][j][3]



            max_score = max(t1, t2, t3, 0)
            score_mat[i][j] = max_score

            if max_score > 0:
                if max_score == t1:
                    direc_mat[i][j][2] = 1


                elif max_score == t2:
                    # horizental
                    direc_mat[i][j][1]=1
                    direc_mat[i][j][4]=E
                else:
                    # vertical, seq1 has a gap
                    direc_mat[i][j][0]=1
                    direc_mat[i][j][3] = E

There are two functions called print_pathpair() and recal() below. They are both used in the traceback function
- print_pathpair() use to print the path in a more formative way


In [34]:
def print_pathpair(alist):
    for path in alist:
        mark1=[]
        mark2=[]
        for i in range(len(path[0])):
            if path[0][i]!='-' and path[1][i]!='-':
                if path[0][i] == path[1][i]:
                    mark1.append(':')

                elif mat1.get(path[0][i],path[1][i])>0:
                    mark1.append('.')

                else:
                    mark1.append(' ')

            else:
                mark1.append(' ')


        path_u = ''.join(path[0])
        path_m = ''.join(mark1)


        path_d = ''.join(path[1])


        print(path_u)
        print(path_m)
        print(path_d)

 ## recal()
 recal(), this function do the recalculation after we find the path.
    1. first the traceback function will store the coordinates of the path node in a list
    2. then for each pair of coordinates , do the recalculation in a recursive way.

In [35]:
def recal(i,j,mat=mat1):
    score_mat[i][j] = 0
    if i< len(seq1) and j <len(seq2):
        if direc_mat[i+1][j][0]==0 and direc_mat[i][j+1][1]==0 and direc_mat[i+1][j+1][2]==0:
            return
    if i< len(seq1):
        i = i+1
        if direc_mat[i][j][0]==1:
            # do recalculate
            t1 = score_mat[i - 1][j - 1] + mat.get(seq1[i - 1], seq2[j - 1])
            # [s(i,j-n(gap)+g(gap]

            t2 = score_mat[i][j - 1] + direc_mat[i][j - 1][4]

            t3 = score_mat[i - 1][j] + direc_mat[i - 1][j][3]

            max_score = max(t1, t2, t3, 0)
            score_mat[i][j] = max_score

            if max_score > 0:
                if max_score == t1:
                    direc_mat[i][j][2] = 1
                    direc_mat[i][j][4] = E

                elif max_score == t2:
                    # horizental
                    direc_mat[i][j][1] = 1
                    direc_mat[i][j][3] = E
                else:
                    # vertical, seq1 has a gap
                    direc_mat[i][j][0] = 1
            recal(i,j)


        i = i-1

    if j < len(seq2):
        j = j+1
        if direc_mat[i][j][1] ==1:
            t1 = score_mat[i - 1][j - 1] + mat.get(seq1[i - 1], seq2[j - 1])

            # [s(i,j-n(gap)+g(gap]

            t2 = score_mat[i][j - 1] + direc_mat[i][j - 1][4]

            t3 = score_mat[i - 1][j] + direc_mat[i - 1][j][3]

            max_score = max(t1, t2, t3, 0)
            score_mat[i][j] = max_score

            if max_score > 0:
                if max_score == t1:
                    direc_mat[i][j][2] = 1
                    direc_mat[i][j][4] = E

                elif max_score == t2:
                    # horizental
                    direc_mat[i][j][1] = 1
                    direc_mat[i][j][3] = E
                else:
                    # vertical, seq1 has a gap
                    direc_mat[i][j][0] = 1
            recal(i, j)
        j = j-1

    if i<len(seq1) and j<len(seq2):
        i = i+1
        j = j+1
        if direc_mat[i][j][2] ==1:
            t1 = score_mat[i - 1][j - 1] + mat.get(seq1[i - 1], seq2[j - 1])

            # [s(i,j-n(gap)+g(gap]

            t2 = score_mat[i][j - 1] + direc_mat[i][j - 1][4]

            t3 = score_mat[i - 1][j] + direc_mat[i - 1][j][3]

            max_score = max(t1, t2, t3, 0)
            score_mat[i][j] = max_score

            if max_score > 0:
                if max_score == t1:
                    direc_mat[i][j][2] = 1
                    direc_mat[i][j][4] = E

                elif max_score == t2:
                    # horizental
                    direc_mat[i][j][1] = 1
                    direc_mat[i][j][3] = E
                else:
                    # vertical, seq1 has a gap
                    direc_mat[i][j][0] = 1
            recal(i, j)


        i = i -1
        j = j -1

## Local Trace
1. the parameter is the maxmum amount of multiple paths
2. how to work
   1. find the largest score in the score_mat
   2. do traceback
   3. for all the path node, do recalculation


In [36]:
def local_trace(k=3):
    # find the largest position(i,j)
    row_large =[]
    for i in range(len(score_mat)):
        row_max = max(score_mat[i])
        index = score_mat[i].index(row_max)
        row_large.append([row_max,index])

    t = [row_large[i][0] for i in range(len(row_large))]
    max_score = max(t)
    i = t.index(max_score)
    j = row_large[i][1]

    print("the cordinate of the starting cordinate:",i,j)

    # i,j is the index of the largest element

    path_pair = []
    path_up = []
    path_down = []


    queue = []
    queue.append([path_up[0:len(path_up)], path_down[0:len(path_down)], i, j])

    recal_pair=[]

    print("score=",score_mat[i][j])
    while (len(queue) > 0):
        t = queue.pop(0)
        i = t[2]
        j = t[3]
        path_up = t[0]
        path_down = t[1]

        if score_mat[i][j] == 0:
            path_pair.append([path_up,path_down])
        # scan all possible path and append it to the queue
        else:
            if direc_mat[i][j][0] == 1:
                path_up.insert(0, '-')
                j = j - 1

                path_down.insert(0, seq2[j])
                recal_pair.append([i,j])
                if len(queue) < k:
                    queue.append([path_up[0:len(path_up)], path_down[0:len(path_down)], i, j])
                path_up = path_up[1:]
                path_down = path_down[1:]
                j = j + 1
            if direc_mat[i][j][1] == 1:
                path_down.insert(0, '-')
                i = i - 1
                path_up.insert(0, seq1[i])
                recal_pair.append([i,j])
                if len(queue) < k:
                    queue.append([path_up[0:len(path_up)], path_down[0:len(path_down)], i, j])
                i = i + 1
                path_up = path_up[1:]
                path_down = path_down[1:]

            if direc_mat[i][j][2] == 1:
                i = i - 1
                j = j - 1
                path_up.insert(0, seq1[i])
                path_down.insert(0, seq2[j])
                recal_pair.append([i,j])
                if len(queue) < k:
                    queue.append([path_up[0:len(path_up)], path_down[0:len(path_down)], i, j])
                i = i + 1
                j = j + 1
                path_up = path_up[1:]
                path_down = path_down[1:]

        print_pathpair(path_pair)

# print("recal_pair:",recal_pair)

    for k in range(len(recal_pair)-1,-1,-1):
        t=recal_pair[k]
        recal(recal_pair[k][0],recal_pair[k][1])
        


In [37]:
# this function is a encapsulation of the local_trace(K)
# the first parameter n specify how many local alignment that would like to find, the K is the maximum multiple subpath.
def multi_local_trace(n,k):
    for i in range(n):
        local_trace(k)

In [38]:
local_score(mat1)

In [39]:
multi_local_trace(3,4)

the cordinate of the starting cordinate: 204 34
score= 191
VPLPAGWEMAKTSSGQRYFLNHIDQTTTWQDPRK
::::::::::::::::::::::::::::::::::
VPLPAGWEMAKTSSGQRYFLNHIDQTTTWQDPRK
the cordinate of the starting cordinate: 262 33
score= 114
PLPDGWEQAMTQDGEIYYINHKNKTTSWLDPR
::: ::: : :  :. :..:: ..::.: :::
PLPAGWEMAKTSSGQRYFLNHIDQTTTWQDPR
the cordinate of the starting cordinate: 425 28
score= 29
FLNSVDEMDT
::: .:.  :
FLNHIDQTTT


![](https://ww2.sinaimg.cn/large/006tNbRwgy1fdo4v0i5l3j30zc0oyjvt.jpg)

The result is the same with that in [LALIGN](http://embnet.vital-it.ch/software/LALIGN_form.html)